In [22]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import os, sys
import wandb
import numpy as np
from tqdm.notebook import tqdm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Add the directory containing the utils folder to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.testing import *
from utils.compatibility import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
all_model_runs = access_wandb_runs(filters=None)

# Baseline
baseline_run = None
for run in all_model_runs: # Hacky
    if run.name == "no-ablation-baseline":
        baseline_run = run
        break

all_model_runs = list(access_wandb_runs())
all_model_runs.append(baseline_run)

In [25]:
def get_l1_sparsity(model):
    """
    Calculate L1 sparsity skipping embedding matrices (wte, wpe)
    """
    total_params = 0
    l1_sum = 0.0
    
    for name, param in model.named_parameters():
        # Skip embedding matrices
        if 'wte' in name or 'wpe' in name:
            continue
            
        total_params += param.numel()
        l1_sum += torch.sum(torch.abs(param.data)).item()
    
    # Normalize by parameter count
    l1_sparsity = (l1_sum / total_params)
    print(f"Total params: {total_params}")
    return l1_sparsity

for run in tqdm(all_model_runs, desc="Processing runs"):
    
    model_dir = f"../model_weights/{run.name}"
    
    ablated_trained_model = load_our_model(model_dir, device=device)
    hooked_model = convert_model_to_hooked_transformer(ablated_trained_model)
    
    sparsity = get_l1_sparsity(hooked_model)
    print(f"Sparsity for {run.name}: {sparsity}")
    update_run(run, {"l1_sparsity": sparsity})


/mnt/c/Dev Projects/Projects/self-ablating-transformers/utils/testing.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=d

Total params: 14535249
Sparsity for fanciful-fog-78: 0.5079001230888454
Total params: 14535249
Sparsity for earnest-moon-79: 0.49274002756728513
Total params: 14535249
Sparsity for super-violet-80: 0.4491382072778325
Total params: 14535249
Sparsity for upbeat-glitter-83: 0.4840176044147763
Total params: 14535249
Sparsity for comfy-cherry-84: 0.5096039813089864
Total params: 14535249
Sparsity for cosmic-leaf-81-part2: 0.7264284916635614
Total params: 14535249
Sparsity for major-planet-86-part3: 0.5497045706686333
Total params: 14535249
Sparsity for light-morning-92: 0.48502937303143484
Total params: 14535249
Sparsity for deft-dew-98-part2: 0.4616967345049077
Total params: 14535249
Sparsity for solar-vortex-103: 0.6705695693077797
Total params: 14535249
Sparsity for peachy-paper-104: 0.4833880563181032
Total params: 14535249
Sparsity for valiant-music-105: 0.4722006684907407
Total params: 14535249
Sparsity for usual-silence-107: 0.7088659383143451
Total params: 14535249
Sparsity for ambe

Welp. Time to rewrite the paper